In [1]:
print("radhe radhe")

radhe radhe


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

# IMPORT DATASET PATH
DATASET_PATH= os.getenv("DATASET_PATH")

In [3]:
import pandas as pd
df=pd.read_csv(DATASET_PATH,index_col="Date",parse_dates=["Date"],dayfirst=False)
df

,Bankura_temp,Birbhum_temp,Burdwan_temp,Darjeeling_temp,Hooghly_temp,Howrah_temp,Jalpaiguri_temp,Kochbihar_temp,Kolkata_temp,Malda_temp,Medinipur_temp,Murshidabad_temp,Nadia_temp,North-24-Parganas_temp,North-Dinajpur_temp,Purulia_temp,South-24-Parganas_temp,South-Dinajpur_temp
Date,,,,,,,,,,,,,,,,,,
1901-01-01,17.970,17.405,18.108,14.058,18.897,19.161,15.593,15.917,19.106,15.670,18.889,17.540,18.428,18.522,15.151,17.159,15.968,16.580
1901-02-01,21.334,21.435,21.879,16.913,22.437,22.497,18.650,19.243,22.600,19.754,21.892,21.501,22.215,21.830,18.845,20.297,18.558,20.409
1901-03-01,26.523,26.422,26.913,21.394,27.463,27.343,23.034,23.535,27.495,24.729,26.686,26.360,26.938,26.097,23.732,25.467,22.129,25.228
1901-04-01,31.712,31.807,31.968,25.806,31.886,31.445,27.258,27.823,31.686,30.027,30.972,31.581,31.725,29.794,28.800,30.974,25.039,30.325
1901-05-01,32.202,31.714,31.934,26.155,31.622,31.316,27.323,27.809,31.392,29.840,31.223,31.243,31.368,29.616,28.859,32.114,25.069,29.959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002-08-01,28.106,28.499,28.547,27.061,28.633,28.567,28.438,29.013,28.697,28.286,27.852,28.819,28.898,27.609,28.474,27.416,23.336,29.026
2002-09-01,28.072,28.519,28.568,26.502,28.653,28.603,27.949,28.511,28.700,28.224,27.888,28.916,29.102,27.743,28.198,27.325,23.492,29.013
2002-10-01,26.541,27.130,27.294,25.052,27.605,27.644,26.433,26.938,27.800,26.828,26.773,27.597,27.991,26.991,26.759,25.715,22.852,27.534


In [8]:

# univariate lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from sklearn.metrics import r2_score


def split_sequence(sequence,n_step):
    X, y = [],[]
    for i in range(len(sequence)):
        end_ix = i + n_step
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)




import datetime
from dateutil.relativedelta import relativedelta  # Import relativedelta

# Define the train end date
train_end_date = datetime.datetime(1997, 12, 1)

# Calculate the test start date by adding 1 month
test_start = train_end_date + relativedelta(months=1)

train_data=df[:train_end_date]
test_data=df[test_start:]

for col in df.columns:



    n_steps=12
    X_train,y_train=split_sequence(train_data[col],n_step=n_steps)
    X_test,y_test=split_sequence(test_data[col],n_step=n_steps)

        
    n_features=1
    X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],n_features)
    X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],n_features)


    model = Sequential()
    model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=20, verbose=1)

    # x_input = array([[60, 70, 80]])
    # x_input = x_input.reshape((1, n_steps, n_features))
    pred = model.predict(X_test, verbose=2)
    r2 = r2_score(y_test, pred)
    print(f"R² Score for {col}: {r2:.4f}")
    # print(yhat)
    # print(x_input.shape)

Epoch 1/20


/tmp/ipykernel_4293/1746719671.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]


36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 74.5776
Epoch 2/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2.3480
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.8937
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.4451
Epoch 5/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.4013
Epoch 6/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.7108
Epoch 7/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.4118
Epoch 8/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.1848
Epoch 9/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9751
Epoch 10/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0106
Epoch 11/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9704
Epoch 12/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.8541
Epoch 13/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.1019
Epoch 14/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9025
Epoch 15/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0066
Epoch 16/20
36/36 ━━━━━━━━━━━

/tmp/ipykernel_4293/1746719671.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
/home/radhe/Subha/internship/werather_forcastiing/venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 241.0513
Epoch 2/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 7.7518
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.9437
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.6045
Epoch 5/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.2728
Epoch 6/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0978
Epoch 7/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.5381
Epoch 8/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.2419
Epoch 9/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.5529
Epoch 10/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9965
Epoch 11/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0499
Epoch 12/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9901
Epoch 13/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0195
Epoch 14/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0540
Epoch 15/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0952
Epoch 16/20
36/36 ━━━━━━━━━━

...

/home/radhe/Subha/internship/werather_forcastiing/venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 179.4765
Epoch 2/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 10.9239
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.1559
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.4324
Epoch 5/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.4323
Epoch 6/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.1221
Epoch 7/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9708
Epoch 8/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9942
Epoch 9/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9179
Epoch 10/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9736
Epoch 11/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.8786
Epoch 12/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.8434
Epoch 13/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9958
Epoch 14/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0058
Epoch 15/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.8545
Epoch 16/20
36/36 ━━━━━━━━━

/tmp/ipykernel_4293/1746719671.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
/home/radhe/Subha/internship/werather_forcastiing/venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 335.8406
Epoch 2/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 14.9362
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.3116
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.0137
Epoch 5/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9578
Epoch 6/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9132
Epoch 7/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9144
Epoch 8/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.7712
Epoch 9/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9238
Epoch 10/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9363
Epoch 11/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.8211
Epoch 12/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.1396
Epoch 13/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.7831
Epoch 14/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.8277
Epoch 15/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.7927
Epoch 16/20
36/36 ━━━━━━━━━

KeyboardInterrupt: 

In [36]:
import datetime
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import Callback
from sklearn.metrics import r2_score

# Example DataFrame
# date_rng = pd.date_range(start='1980-01-01', end='2000-01-01', freq='MS')
# df = pd.DataFrame(np.random.randn(len(date_rng), 1), index=date_rng, columns=['Value'])

def split_sequence(sequence, n_steps):
    X, y = [], []
    for i in range(len(sequence) - n_steps):
        seq_x, seq_y = sequence[i:i+n_steps], sequence[i+n_steps]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Custom Callback to Stop Training When Loss < 0.6
class StopAtLossThreshold(Callback):
    def __init__(self, threshold=0.6):
        super(StopAtLossThreshold, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        if logs.get("loss") is not None and logs["loss"] < self.threshold:
            print(f"\nStopping training as loss reached {logs['loss']:.4f} (below {self.threshold})")
            self.model.stop_training = True

# Define the train-test split based on dates
train_end_date = datetime.datetime(1997, 12, 1)
test_start = train_end_date + relativedelta(months=1)

train_data = df.loc[:train_end_date]
test_data = df.loc[test_start:]

n_steps = 12
n_features = 1

for col in df.columns:
    X_train, y_train = split_sequence(train_data[col].values, n_steps)
    X_test, y_test = split_sequence(test_data[col].values, n_steps)

    # Reshape input for LSTM model
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))

    # Define LSTM Model
    model = Sequential([
        LSTM(100, activation='relu', input_shape=(n_steps, n_features), return_sequences=True),
        # Dropout(0.3),
        LSTM(50, activation='relu'),
        # Dropout(0.5),
        
        # LSTM(50, activation='relu'),
        Dense(1)
    ])
    
    model.compile(optimizer='adam', loss='mse')

    # Train model with custom callback
    model.fit(X_train, y_train, epochs=30, verbose=1, callbacks=[StopAtLossThreshold(threshold=0.8)])

    # Make predictions
    pred = model.predict(X_test, verbose=0)
    
    # Evaluate performance
    r2 = r2_score(y_test, pred)
    print(f"R² Score for {col}: {r2:.4f}")


Epoch 1/30


36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 336.3685
Epoch 2/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 9.5986
Epoch 3/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 2.0010
Epoch 4/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.4674
Epoch 5/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.1054
Epoch 6/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0795
Epoch 7/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9640
Epoch 8/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.0697
Epoch 9/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.1141
Epoch 10/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.0152
Epoch 11/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.0575
Epoch 12/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.8779
Epoch 13/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.0033
Epoch 14/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.2988
Epoch 15/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.9078
Epoch 16/30
36/36 ━━━━━━

/home/radhe/Subha/internship/werather_forcastiing/venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 133.1724
Epoch 2/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 4.5899
Epoch 3/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.2266
Epoch 4/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.6003
Epoch 5/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.3177
Epoch 6/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.2100
Epoch 7/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.4793
Epoch 8/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.5628
Epoch 9/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9715
Epoch 10/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.0128
Epoch 11/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.1378
Epoch 12/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.8794
Epoch 13/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.9394
Epoch 14/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9916
Epoch 15/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9479
Epoch 16/30
36/36 ━━━━━━━━━━

KeyboardInterrupt: 

In [20]:
import datetime
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.callbacks import Callback, EarlyStopping
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler

# Example DataFrame (Replace with actual data)
# date_rng = pd.date_range(start='1980-01-01', end='2000-01-01', freq='MS')
# df = pd.DataFrame(np.random.randn(len(date_rng), 1), index=date_rng, columns=['Value'])

# Normalize Data for Better LSTM Performance


# Function to Split Sequence into Samples
def split_sequence(sequence, n_steps):
    X, y = [], []
    for i in range(len(sequence) - n_steps):
        seq_x, seq_y = sequence[i:i+n_steps], sequence[i+n_steps]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Custom Callback to Stop Training When Loss < 0.6
class StopAtLossThreshold(Callback):
    def __init__(self, threshold=0.6):
        super(StopAtLossThreshold, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        if logs.get("loss") is not None and logs["loss"] < self.threshold:
            print(f"\nStopping training as loss reached {logs['loss']:.4f} (below {self.threshold})")
            self.model.stop_training = True

# Define Train-Test Split
train_end_date = datetime.datetime(1997, 12, 1)
test_start = train_end_date + relativedelta(months=1)

train_data = df.loc[:train_end_date]
test_data = df.loc[test_start:]

n_steps = 20  # Increased to capture long-term trends
n_features = 1

for col in df.columns:
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    df_scaled = pd.DataFrame(scaler.fit_transform(df[col]), index=df.index, columns=df.columns)
    train_data = df_scaled.loc[:train_end_date]
    test_data = df_scaled.loc[test_start:]
    
    X_train, y_train = split_sequence(train_data.values, n_steps)
    X_test, y_test = split_sequence(test_data.values, n_steps)

    # Reshape Input for LSTM
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))

    # Define Optimized LSTM Model
    model = Sequential([
        Bidirectional(LSTM(128, activation='relu', return_sequences=True), input_shape=(n_steps, n_features)),
        Dropout(0.3),  # Increase dropout to avoid overfitting
        LSTM(64, activation='relu', return_sequences=True),
        Dropout(0.2),
        LSTM(32, activation='relu'),
        Dense(1)
    ])
    
    model.compile(optimizer='adam', loss='mse')

    # Train Model with Early Stopping
    early_stop = EarlyStopping(monitor='loss', patience=20, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=300, batch_size=16, verbose=1, callbacks=[StopAtLossThreshold(0.6), early_stop])

    # Make Predictions and Inverse Transform
    pred = model.predict(X_test, verbose=0)
    pred_original = scaler.inverse_transform(pred.reshape(-1, 1))
    y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Evaluate Performance
    r2 = r2_score(y_test_original, pred_original)
    print(f"R² Score for {col}: {r2:.4f}")


ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.

In [21]:
import datetime
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.callbacks import Callback, EarlyStopping
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler

# Example DataFrame (Replace with actual data)
# date_rng = pd.date_range(start='1980-01-01', end='2000-01-01', freq='MS')
# df = pd.DataFrame(np.random.randn(len(date_rng), 1), index=date_rng, columns=['Value'])

# Normalize Data for Better LSTM Performance
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = pd.DataFrame(scaler.fit_transform(df), index=df.index, columns=df.columns)

# Function to Split Sequence into Samples
def split_sequence(sequence, n_steps):
    X, y = [], []
    for i in range(len(sequence) - n_steps):
        seq_x, seq_y = sequence[i:i+n_steps], sequence[i+n_steps]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Custom Callback to Stop Training When Loss < 0.6
class StopAtLossThreshold(Callback):
    def __init__(self, threshold=0.6):
        super(StopAtLossThreshold, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        if logs.get("loss") is not None and logs["loss"] < self.threshold:
            print(f"\nStopping training as loss reached {logs['loss']:.4f} (below {self.threshold})")
            self.model.stop_training = True

# Define Train-Test Split
train_end_date = datetime.datetime(1997, 12, 1)
test_start = train_end_date + relativedelta(months=1)

train_data = df_scaled.loc[:train_end_date]
test_data = df_scaled.loc[test_start:]

n_steps = 20  # Increased to capture long-term trends
n_features = 1

for col in df_scaled.columns:
    X_train, y_train = split_sequence(train_data[col].values, n_steps)
    X_test, y_test = split_sequence(test_data[col].values, n_steps)

    # Reshape Input for LSTM
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))

    # Define Optimized LSTM Model
    model = Sequential([
        Bidirectional(LSTM(128, activation='relu', return_sequences=True), input_shape=(n_steps, n_features)),
        Dropout(0.3),  # Increase dropout to avoid overfitting
        LSTM(64, activation='relu', return_sequences=True),
        Dropout(0.2),
        LSTM(32, activation='relu'),
        Dense(1)
    ])
    
    model.compile(optimizer='adam', loss='mse')

    # Train Model with Early Stopping
    early_stop = EarlyStopping(monitor='loss', patience=20, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=300, batch_size=16, verbose=1, callbacks=[StopAtLossThreshold(0.6), early_stop])

    # Make Predictions and Fix Shape Issue for Inverse Transform
    pred = model.predict(X_test, verbose=0)

    # Ensure inverse transformation applies only to one column
    pred_original = scaler.inverse_transform(np.hstack([pred, np.zeros((pred.shape[0], df_scaled.shape[1] - 1))]))[:, 0]
    y_test_original = scaler.inverse_transform(np.hstack([y_test.reshape(-1, 1), np.zeros((y_test.shape[0], df_scaled.shape[1] - 1))]))[:, 0]

    # Evaluate Performance
    r2 = r2_score(y_test_original, pred_original)
    print(f"R² Score for {col}: {r2:.4f}")


Epoch 1/300


/home/radhe/Subha/internship/werather_forcastiing/venv/lib/python3.10/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1458
Stopping training as loss reached 0.0924 (below 0.6)
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - loss: 0.1451
R² Score for Bankura_temp: -0.0511
Epoch 1/300


/home/radhe/Subha/internship/werather_forcastiing/venv/lib/python3.10/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


71/72 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1422
Stopping training as loss reached 0.0908 (below 0.6)
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.1408
R² Score for Birbhum_temp: -0.1578
Epoch 1/300


...

70/72 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.1439
Stopping training as loss reached 0.0935 (below 0.6)
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - loss: 0.1418
R² Score for Purulia_temp: -0.1628
Epoch 1/300


/home/radhe/Subha/internship/werather_forcastiing/venv/lib/python3.10/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1582
Stopping training as loss reached 0.0987 (below 0.6)
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.1574
R² Score for South-24-Parganas_temp: -0.0997
Epoch 1/300


/home/radhe/Subha/internship/werather_forcastiing/venv/lib/python3.10/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


71/72 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1404
Stopping training as loss reached 0.0964 (below 0.6)
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.1392
R² Score for South-Dinajpur_temp: -0.0803
